In [1]:
import requests
from typing import List, Tuple, Union, Optional

In [2]:
from scrapy_monit_web.monitor.scrapyd_api import *

In [5]:
r = get_scrapyd_logs("http://185.250.148.161", 'default','smartelectronics_items', '46bef732f4ba11ed8ad9ed526d4a5438')
print(r)

2023-05-17 18:21:50 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: price_monitoring)
2023-05-17 18:21:50 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.8.10 (default, Mar 15 2022, 12:22:08) - [GCC 9.4.0], pyOpenSSL 22.0.0 (OpenSSL 1.1.1n  15 Mar 2022), cryptography 36.0.2, Platform Linux-5.4.0-109-generic-x86_64-with-glibc2.29
2023-05-17 18:21:50 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'BOT_NAME': 'price_monitoring',
 'CONCURRENT_REQUESTS': '1',
 'DOWNLOAD_DELAY': '1',
 'DUPEFILTER_CLASS': 'scrapy.dupefilters.BaseDupeFilter',
 'LOG_FILE': 'logs/default/smartelectronics_items/46bef732f4ba11ed8ad9ed526d4a5438.log',
 'NEWSPIDER_MODULE': 'price_monitoring.spiders',
 'SPIDER_MODULES': ['price_monitoring.spiders'],
 'USER_AGENT': 'Mozilla/5.0 (compatible; YandexAccessibilityBot/3.0; '
               '+http://yandex.com/bots)'}
2023-05-17 18:21:50 [scrapy.utils.log]

In [ ]:
def subdict(_dict: dict, keys: Union[list, tuple]):
    return {k: _dict[k] for k in keys}


def api_daemon_status():
    url = "http://185.250.148.161/daemonstatus.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        fields = ["status","running", "pending", "finished", "node_name"]
        return subdict(data, fields)
    else:
        return None
    

def api_list_projects():
    url = "http://185.250.148.161/listprojects.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['projects']
    else:
        return None
    

def api_listversions(project: str='default'):
    """  """
    url = "http://185.250.148.161/listversions.json"
    # check if project in projects
    assert project in api_list_projects()
    params = {'project': project}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data['versions']
    else:
        return None


def api_listspiders(project: str='default', _version: str=None):
    """  """
    url = "http://185.250.148.161/listspiders.json"
    # check if project in projects
    assert project in api_list_projects()
    if not _version is None:
        assert _version in api_listversions(project)
    params = {'project': project, '_version': _version}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data['spiders']
    else:
        return None
    

def api_listjobs(project: str='default'):
    """ 
        {
            "status": "ok",
            "pending": [
                {
                    "project": "myproject", "spider": "spider1",
                    "id": "78391cc0fcaf11e1b0090800272a6d06"
                }
            ],
            "running": [
                {
                    "id": "422e608f9f28cef127b3d5ef93fe9399",
                    "project": "myproject", "spider": "spider2",
                    "start_time": "2012-09-12 10:14:03.594664"
                }
            ],
            "finished": [
                {
                    "id": "2f16646cfcaf11e1b0090800272a6d06",
                    "project": "myproject", "spider": "spider3",
                    "start_time": "2012-09-12 10:14:03.594664",
                    "end_time": "2012-09-12 10:24:03.594664",
                    "log_url": "/logs/myproject/spider3/2f16646cfcaf11e1b0090800272a6d06.log",
                    "items_url": "/items/myproject/spider3/2f16646cfcaf11e1b0090800272a6d06.jl"
                }
            ]
        }
    """
    url = "http://185.250.148.161/listjobs.json"
    # check if project in projects
    assert project in api_list_projects()

    params = {'project': project}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return None


def api_delversion(_version: str, project: str='default'):
    """  """
    url = "http://185.250.148.161/delversion.json"
    # check if project in projects
    assert project in api_list_projects()
    assert _version in api_listversions(project)

    params = {'project': project, '_version': _version}
    response = requests.post(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data['status']
    else:
        return None
    

def api_delproject(project: str):
    """  """
    url = "http://185.250.148.161/delproject.json"
    # check if project in projects
    assert project in api_list_projects()
    params = {'project': project}
    response = requests.post(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data['status']
    else:
        return None
    

def get_scrapyd_logs(project_name: str, spider_name: str, job_id: str):
    """  """
    # http://185.250.148.161; default; wt_list; f205c3d4f64311ed8ad9ed526d4a5438
    api_url = 'http://185.250.148.161'
    url = f"{api_url}/logs/{project_name}/{spider_name}/{job_id}.log"
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        raise Exception(f"Exception: status code got: {response.status_code}")
    



def run_scrapy_spider(spider_name: str, project_name: str='default'):
    """  """
    assert spider_name in api_listspiders(project=project_name)

    api_url = 'http://185.250.148.161'
    schedule_url = f'{api_url}/schedule.json'
    response = requests.post(schedule_url, data={'project': project_name, 'spider': spider_name})
    if response.status_code == 200:
        print('Spider scheduled successfully.')
    else:
        print('Spider scheduling failed.')


run_scrapy_spider('bestel_list')